In [1]:
%%time
from algoutils import *

raw_courses = read('data/courses.txt')
raw_industries = read('data/industries.txt')
raw_job_industries = read('data/job_industries.txt')

CPU times: user 4 ms, sys: 8 ms, total: 12 ms
Wall time: 9.66 ms


In [2]:
%%time
import cleaning
reload(cleaning)
# note: enchant module and library should be installed
from cleaning import clean

clean_courses = clean(raw_courses, style='course')
clean_industries = clean(raw_industries, style='industry')
clean_job_industries = clean(raw_job_industries, style='job title')

CPU times: user 18.6 s, sys: 164 ms, total: 18.8 s
Wall time: 18.7 s


In [3]:
%%time
# note: nltk module and nltk.wordnet corpus should be installed
from clustering import stem_cluster
 
course_clusters = stem_cluster(clean_courses)
industry_clusters = stem_cluster(clean_industries)
job_industry_clusters = stem_cluster(clean_job_industries, mode=5)

CPU times: user 76 ms, sys: 8 ms, total: 84 ms
Wall time: 82 ms


In [4]:
sorted(course_clusters)[:10]

['ACCOUNTANCY',
 'ACCOUNTING',
 'ADMINISTRATION',
 'ADVERTISING',
 'AGRIBUSINESS',
 'AGRICULTURAL',
 'AGRICULTURE',
 'ANIMATION',
 'APPLICATION',
 'APPLIED']

In [5]:
sorted(industry_clusters)[:10]

['ACCOUNTING',
 'ADVERTISING',
 'AGENCY',
 'ART',
 'ASSISTANT',
 'AUTOMOTIVE',
 'BANKING',
 'BEAUTY',
 'BEVERAGE',
 'BPO']

In [6]:
sorted(job_industry_clusters)[:10]

['BUSINESS',
 'CALL',
 'CENTER',
 'COMPUTER',
 'CONSTRUCTION',
 'DEPARTMENT',
 'DESIGN',
 'DEVELOPMENT',
 'DISTRIBUTION',
 'EDUCATION']

In [7]:
%%time

import analysis
reload(analysis)
from analysis import *

# note: pandas and sklearn modules should be installed
# initializes a likelihood data frame based on row and column labels.
# Performs a content-based filtering technique using features.
# vectorizer can be 'tfidf' or 'count'

CourseIndustry = LikelihoodMatrix(course_clusters, industry_clusters, vectorizer='tfidf', ngram_range=(3,4))

CPU times: user 220 ms, sys: 40 ms, total: 260 ms
Wall time: 656 ms


In [8]:
CourseIndustry.dataframe.head()

,ENGINE,REPAIR,CONTRACTOR,HARDWARE,PRODUCT,PART,HOSPITALITY,LINE,CAFE,CUSTOMER,...,CITY,PLANT,ART,PUBLISHING,SYSTEM,FABRICATION,HEALTH,ASSISTANT,DISTRIBUTOR,BROKERAGE
LINGUISTICS,0,0,0,0,0,0,0,0.112283,0,0,...,0.000000,0,0,0.020794,0,0,0,0.045424,0.042784,0
HARDWARE,0,0,0,1,0,0,0,0.000000,0,0,...,0.000000,0,0,0.000000,0,0,0,0.000000,0.000000,0
HOSPITALITY,0,0,0,0,0,0,1,0.000000,0,0,...,0.108723,0,0,0.000000,0,0,0,0.000000,0.000000,0
ELEMENTARY,0,0,0,0,0,0,0,0.000000,0,0,...,0.000000,0,0,0.000000,0,0,0,0.000000,0.000000,0
NCII,0,0,0,0,0,0,0,0.000000,0,0,...,0.000000,0,0,0.000000,0,0,0,0.000000,0.000000,0


In [9]:
%%time

# note: pandas and sklearn modules should be installed
# derives the change in likelihood if an entry

# computes delta likelihood using the following steps: (non-verbatim)
# clabels = df.column_labels
# rlabels = df.row_labels
# vr = [cos_similarity(ur, label) for label in rlabels]
# vc = [cos_similarity(vr, label) for label in clabels]
# dL = vr * vc^T

d1 = CourseIndustry.delta('CALLCENTE', 'BUSINESS')
d2 = CourseIndustry.delta('MARKETING', 'RESEARCH & DEVELOPMENT')
d3 = CourseIndustry.delta('AGRICULTURE', 'FINANCE')

CPU times: user 100 ms, sys: 0 ns, total: 100 ms
Wall time: 105 ms


In [10]:
%%time

# dynamic update of likelihood matrix
# just accumulate small likelihoods
# accounts for relative likelihood and not absolute percentage

users = [
    ('COMPUTER SCIENCE', 'INFORMATION TECHNOLOGY'),
    ('ACCOUNTING', 'BUSINESS'),
    ('BUSINESS ADMINSTRATION', 'FINANCE'),
    ('MARKETING', 'FINANCE'),
    ('MATHEMATICS', 'FINANCE'),
    ('SOCIAL SCIENCE', 'CONSULATION'),
    ('LITERATURE', 'WRITING'),
    ('MANAGEMENT', 'BANKING'),
    ('SCIENCES', 'RESEARCH')
]

for course, industry in users:
    CourseIndustry.add_match(course, industry)

CPU times: user 208 ms, sys: 0 ns, total: 208 ms
Wall time: 208 ms


In [11]:
# get specific likelihood coefficient of a pair of clusters
# e.g. likelihood of courses from the BUSINESS industry

# note: 1.00 does not mean 100%
# likelihood metric is relative
# the more data, the more accurate the prediction

%time CourseIndustry.dataframe.BUSINESS.nlargest(10)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 776 µs


BUSINESS        1.000000
ACCOUNTING      1.000000
AGRIBUSINESS    0.754798
ACCOUNTANCY     0.593833
NURSING         0.081065
MARINE          0.072175
ENGINEER        0.059954
MEDICINE        0.059395
ADVERTISING     0.056718
ENGINEERING     0.048657
Name: BUSINESS, dtype: float64

In [12]:
%%time

# transpose data frame for rows
CourseIndustry.dataframe.T.SCIENCE.nlargest(6)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.06 ms


RESEARCH       0.952395
INFORMATION    0.497331
TECHNOLOGY     0.463501
CONSULTING     0.410553
CONSULTANCY    0.368664
AGENCY         0.119928
Name: SCIENCE, dtype: float64

In [13]:
%%time

# add count from a specific matching pair
# note: this is synonymous to CourseIndustry.dataframe += CourseIndustry.delta('PHYSICS', 'REPAIR SUPPLY')

CourseIndustry.add_match('PHYSICS', 'RESEARCH')

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 16.3 ms


In [14]:
%%time

# returns a vector of recommend likelihood of a course from a specific industry
# since courses are rows, we find column matches
# we can also make the method output percentages

matches = CourseIndustry.find_column_matches('PHYSICS', with_labels = True, percentage = True)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 323 µs


In [15]:
# rank recommendations with the highest percentage
matches.nlargest(5)

RESEARCH      72.576056
MEDICAL        7.875103
CHEMICAL       6.808566
ELECTRICAL     6.062457
CALLCENTER     1.973803
dtype: float64

In [16]:
%%time

# we can also get the recommendation score of a specific pair
# this is equivalent to SV1^T * LM * SV2^T
# where SV is similarity_vector, LM is likelihood matrix

CourseIndustry.recommendation_score('PHYSICS', 'SCIENTIFIC RESEARCH')

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 18.4 ms


1.1563182150589491

In [17]:
%%time

# You can also get the dot product of data frames
# to get bridged likelihood

IndustryJob = LikelihoodMatrix(industry_clusters, job_industry_clusters, vectorizer='tfidf', ngram_range=(3,4))
CourseJob = CourseIndustry * IndustryJob

CPU times: user 64 ms, sys: 4 ms, total: 68 ms
Wall time: 77.4 ms


In [18]:
CourseJob.recommendation_score('PHYSICS', 'MANAGEMENT')

0.0

In [19]:
CourseJob.recommendation_score('COMPUTER SCIENCE', 'ARCHITECTURE')

0.0

In [20]:
CourseJob.recommendation_score('COMPUTER SCIENCE', 'TECHNOLOGY')

0.0

In [21]:
# after adding a match to course and a job, recommendation score should go up
CourseJob.add_match('COMPUTER SCIENCE', 'TECHNOLOGY')
CourseJob.add_match('PROGRAMMING', 'COMPUTER')

In [22]:
CourseJob.recommendation_score('PROGRAMMING', 'COMPUTER')

1.5089329953492852